# ⚡ Performance Optimization Testing

**Goal:** Identify bottlenecks and optimize for speed

## What We're Testing:
1. **Component Latency** - Which parts are slowest?
2. **Token Usage** - Optimize prompt efficiency
3. **Caching Opportunities** - What to cache?
4. **Parallel Processing** - Where can we parallelize?

In [ ]:
from kaelum import enhance
from kaelum.runtime.orchestrator import MCP
from kaelum.core.config import MCPConfig, LLMConfig
import time
import functools

MODEL = "llama3.2:3b"  # Fast model for testing

print(f"✅ Performance testing setup for: {MODEL}")

## Test 1: Component Latency Breakdown

**Measure time spent in each pipeline stage**

In [ ]:
import time

def time_component(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = (time.time() - start) * 1000
        print(f"{func.__name__}: {elapsed:.0f}ms")
        return result
    return wrapper

# Setup MCP with instrumentation
config = MCPConfig(llm=LLMConfig(model=MODEL, max_tokens=512))
mcp = MCP(config)

# Monkey patch to measure timing
original_generate_reasoning = mcp.generator.generate_reasoning
original_verify = mcp.verification.verify_trace

mcp.generator.generate_reasoning = time_component(original_generate_reasoning)
mcp.verification.verify_trace = time_component(original_verify)

query = "What is 15% of 200?"

print(f"Testing query: {query}")
print("\nComponent timing:")
print("="*60)

start_total = time.time()
result = mcp.infer(query)
total_time = (time.time() - start_total) * 1000

print(f"\nTotal: {total_time:.0f}ms")
print(f"\nResult: {result['final']}")

**📝 Latency Breakdown:**
- Reasoning generation: ___ms
- Verification: ___ms
- Reflection (if any): ___ms
- Bottleneck:

## Test 2: Token Usage Analysis

**Measure prompt efficiency - fewer tokens = faster**

In [ ]:
# Test different prompt configurations
test_configs = [
    {"name": "Minimal", "max_tokens": 256},
    {"name": "Standard", "max_tokens": 512},
    {"name": "Detailed", "max_tokens": 1024},
]

query = "Solve: 2x + 5 = 15"

for config in test_configs:
    print(f"\n{'='*60}")
    print(f"Config: {config['name']} (max_tokens={config['max_tokens']})")
    print(f"{'='*60}")
    
    start = time.time()
    result = enhance(
        query,
        mode="math",
        model=MODEL,
        max_tokens=config['max_tokens'],
        max_iterations=1
    )
    elapsed = (time.time() - start) * 1000
    
    print(f"Time: {elapsed:.0f}ms")
    print(f"Response length: {len(result)} chars")
    print(f"Result preview: {result[:150]}...")

**📝 Token Efficiency:**
- Sweet spot for max_tokens:
- Speed vs quality tradeoff:

## Test 3: Temperature Impact

**Lower temperature = faster + more deterministic**

In [ ]:
temperatures = [0.0, 0.3, 0.5, 0.7]
query = "Calculate: 25 × 8"

for temp in temperatures:
    print(f"\n{'='*60}")
    print(f"Temperature: {temp}")
    print(f"{'='*60}")
    
    times = []
    for run in range(3):  # 3 runs to average
        start = time.time()
        result = enhance(query, model=MODEL, temperature=temp, max_iterations=1)
        elapsed = (time.time() - start) * 1000
        times.append(elapsed)
    
    avg_time = sum(times) / len(times)
    print(f"Average time: {avg_time:.0f}ms (across 3 runs)")
    print(f"Sample result: {result[:100]}...")

**📝 Temperature Results:**
- Fastest temperature:
- Quality impact:
- Recommendation:

## Test 4: Batch Processing

**Test multiple queries - identify caching opportunities**

In [ ]:
# Similar queries that could benefit from caching
queries = [
    "What is 10% of 100?",
    "What is 20% of 100?",
    "What is 30% of 100?",
    "What is 10% of 100?",  # Duplicate - should be cached
]

print("Testing batch queries (cache=True)...\n")

for i, query in enumerate(queries, 1):
    print(f"Query {i}: {query}")
    start = time.time()
    result = enhance(query, model=MODEL, cache=True, max_iterations=1)
    elapsed = (time.time() - start) * 1000
    print(f"Time: {elapsed:.0f}ms")
    
    if i == 4:  # Should be faster (cache hit)
        print("👆 This should be instant (cache hit)")
    print()

**📝 Caching Analysis:**
- Did duplicate query use cache?
- Speed improvement:
- What else to cache?

## Test 5: Skip Reflection for Simple Queries

**Test confidence threshold tuning**

In [ ]:
simple_queries = [
    "What is 5 + 5?",
    "What is 10 × 2?",
    "What is 100 ÷ 4?",
]

print("Testing simple queries (should skip reflection)...\n")

for query in simple_queries:
    print(f"Query: {query}")
    start = time.time()
    result = enhance(query, model=MODEL, max_iterations=2)  # Allow reflection
    elapsed = (time.time() - start) * 1000
    
    # Check if reflection was skipped
    import re
    iterations = 0
    match = re.search(r'iterations?:\s*(\d+)', result.lower())
    if match:
        iterations = int(match.group(1))
    
    print(f"Time: {elapsed:.0f}ms")
    print(f"Iterations: {iterations} (0 = skipped reflection ✅)")
    print()

**📝 Reflection Optimization:**
- Are simple queries fast?
- Reflection skipped correctly?
- Threshold tuning needed?

## 🎯 Performance Summary

| Optimization | Current | Target | Status |
|--------------|---------|--------|--------|
| Reasoning latency | ___ms | < 2000ms | ___ |
| Verification overhead | ___ms | < 100ms | ___ |
| Total pipeline | ___ms | < 3000ms | ___ |
| Token efficiency | ___ tokens | < 1000 | ___ |

**Bottlenecks Found:**
1.
2.
3.

**Optimization Recommendations:**
1.
2.
3.

**Next Steps:**
- Implement recommended optimizations
- Add LRU caching for common queries
- Consider async/parallel processing
- Profile with production data